# Recommender System for House using London AirBnB Data
## Module 1

## Load dependencies

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

import re
import sys
import itertools
import warnings
import string
warnings.filterwarnings("ignore")
from urllib.request import urlopen
import pickle

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Data Gathering

In [8]:
listings = pd.read_csv("http://data.insideairbnb.com/united-kingdom/england/london/2021-12-07/data/listings.csv.gz")

In [4]:
#Load processed dataset
data_cl = pickle.load(urlopen("https://fabriziorocco.it/listings_sub.pkl"))
data_cl

,listing_url,name,description,price,amenities_upd,vector
0,https://www.airbnb.com/rooms/13913,holiday london db room let on going,my bright double bedroom with a large window h...,65.00,hot water heating coffee maker building...,"holiday london db room let on going,my bright ..."
1,https://www.airbnb.com/rooms/15400,bright chelsea apartment chelsea,lots of windows and light st luke s gardens ...,75.00,hot water heating coffee maker tv with ...,"bright chelsea apartment chelsea ,lots of wi..."
2,https://www.airbnb.com/rooms/17402,superb bed bath wifi trendy,you ll have a wonderful stay in this superb mo...,265.00,hot water heating coffee maker iron d...,"superb bed bath wifi trendy ,you ll ha..."
3,https://www.airbnb.com/rooms/17506,boutique chelsea fulham double bed star ensuite,enjoy a chic stay in this elegant but fully mo...,150.00,shampoo carbon monoxide alarm hot water ...,boutique chelsea fulham double bed star ensu...
4,https://www.airbnb.com/rooms/25123,clean big room in london room,big room with double bed clean sheets clean ...,29.00,shampoo washer heating coffee maker e...,"clean big room in london room ,big room wit..."
...,...,...,...,...,...,...
66636,https://www.airbnb.com/rooms/53709893,private bedroom crystal palace in high street,no description,120.00,washer carbon monoxide alarm heating ho...,private bedroom crystal palace in high stree...
66637,https://www.airbnb.com/rooms/53710986,chiswick park bed bath chiswick west london,beautiful modern top floor penthouse apartment...,288.00,heating private entrance first aid kit ...,chiswick park bed bath chiswick west lon...
66638,https://www.airbnb.com/rooms/53711668,shabby chic beds bungalow in woodgreen,a mix of modern and antique decor with bed b...,104.00,hot water heating coffee maker private ...,"shabby chic beds bungalow in woodgreen,a mix..."
66639,https://www.airbnb.com/rooms/53712786,outstanding luxury notting hill gate,beautiful light and breathtaking flat situ...,402.00,hot water heating dedicated workspace c...,"outstanding luxury notting hill gate,bea..."


In [9]:
#Have a first look at all the data
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66641 entries, 0 to 66640
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            66641 non-null  int64  
 1   listing_url                                   66641 non-null  object 
 2   scrape_id                                     66641 non-null  int64  
 3   last_scraped                                  66641 non-null  object 
 4   name                                          66617 non-null  object 
 5   description                                   64297 non-null  object 
 6   neighborhood_overview                         39967 non-null  object 
 7   picture_url                                   66641 non-null  object 
 8   host_id                                       66641 non-null  int64  
 9   host_url                                      66641 non-null 

## Preprocessing

### NAs 

In [8]:
listings['description'] = listings['description'].fillna('No description')
listings['neighborhood_overview'] = listings['neighborhood_overview'].fillna('No neighborhood_overview')

### Drop Columns

For this project we're only interested in **price, title, description, url** and **amenities**

In [10]:
listings.drop(listings.columns[8:25], axis=1, inplace=True) # Drop them since useless for the sake of this project
listings.drop(listings.columns[2:3], axis=1, inplace=True) # Drop them since useless for the sake of this project
listings.drop('last_scraped', axis=1, inplace=True) # Drop them since useless for the sake of this project
listings.drop('host_identity_verified', axis=1, inplace=True) # Drop them since useless for the sake of this project
listings.drop('reviews_per_month', axis=1, inplace=True) # Drop them since useless for the sake of this project
listings.drop(listings.columns[23:56], axis=1, inplace=True) # Drop them since useless for the sake of this project

### Regex functions

In [11]:
listings['amenities_upd'] = listings['amenities'].apply(lambda x: [re.sub(' ', '_', i) for i in re.findall(r'"([^"]*)"', x)])
listings['amenities_upd'].values[0][0]

'Hot_water'

In [12]:
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', str(x))
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

In [13]:
listings['name'] = listings.name.map(alphanumeric)
listings.head()

,id,listing_url,name,description,neighborhood_overview,picture_url,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,amenities_upd
0,13913,https://www.airbnb.com/rooms/13913,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,"Islington, Greater London, United Kingdom",Islington,NaN,51.56861,-0.11270,Private room in rental unit,Private room,2,NaN,1 shared bath,1.0,NaN,"[""Hot water"", ""Heating"", ""Coffee maker"", ""Buil...",$65.00,1,29,1.0,1.0,29.0,29.0,1.0,29.0,NaN,t,26,56,86,361,2021-12-09,22,1,1,2010-08-18,2021-11-21,4.86,4.80,4.85,4.80,4.90,4.65,4.75,NaN,f,2,1,1,0,"[Hot_water, Heating, Coffee_maker, Building_st..."
1,15400,https://www.airbnb.com/rooms/15400,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,https://a0.muscache.com/pictures/428392/462d26...,"London, United Kingdom",Kensington and Chelsea,NaN,51.48780,-0.16813,Entire rental unit,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Hot water"", ""Heating"", ""Coffee maker"", ""TV w...",$75.00,3,50,3.0,3.0,50.0,50.0,3.0,50.0,NaN,t,0,0,0,4,2021-12-09,89,0,0,2009-12-21,2020-03-16,4.79,4.84,4.88,4.87,4.82,4.93,4.73,NaN,f,1,1,0,0,"[Hot_water, Heating, Coffee_maker, TV_with_sta..."
2,17402,https://www.airbnb.com/rooms/17402,Superb -Bed/ Bath & Wifi: Trendy,You'll have a wonderful stay in this superb mo...,"Location, location, location! You won't find b...",https://a0.muscache.com/pictures/39d5309d-fba7...,"London, Fitzrovia, United Kingdom",Westminster,NaN,51.52195,-0.14094,Entire rental unit,Entire home/apt,6,NaN,2 baths,3.0,3.0,"[""Hot water"", ""Heating"", ""Coffee maker"", ""Iron...",$265.00,4,365,2.0,4.0,365.0,365.0,4.0,365.0,NaN,t,4,34,60,295,2021-12-09,43,1,0,2011-03-21,2021-06-28,4.69,4.80,4.68,4.66,4.66,4.85,4.59,NaN,f,1,1,0,0,"[Hot_water, Heating, Coffee_maker, Iron, Dryer..."
3,17506,https://www.airbnb.com/rooms/17506,Boutique Chelsea/Fulham Double bed -star ensuite,Enjoy a chic stay in this elegant but fully mo...,Fulham is 'villagey' and residential – a real ...,https://a0.muscache.com/pictures/11901327/e63d...,"London, United Kingdom",Hammersmith and Fulham,NaN,51.47935,-0.19743,Private room in townhouse,Private room,2,NaN,1 private bath,1.0,1.0,"[""Shampoo"", ""Carbon monoxide alarm"", ""Hot wate...",$150.00,3,21,3.0,3.0,21.0,21.0,3.0,21.0,NaN,t,29,59,89,364,2021-12-09,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,2,0,2,0,"[Shampoo, Carbon_monoxide_alarm, Hot_water, He..."
4,25123,https://www.airbnb.com/rooms/25123,Clean big Room in London (Room ),Big room with double bed/ clean sheets/ clean ...,Barnet is one of the largest boroughs in Londo...,https://a0.muscache.com/pictures/456905/a004b9...,"Barnet, England, United Kingdom",Barnet,NaN,51.57438,-0.21081,Private room in residential home,Private room,2,NaN,1.5 shared baths,1.0,NaN,"[""Shampoo"", ""Washer"", ""Heating"", ""Coffee maker...",$29.00,10,1120,10.0,10.0,1125.0,1125.0,10.0,1125.0,NaN,t,0,0,0,0,2021-12-09,129,0,0,2010-04-14,2020-03-15,4.78,4.70,4.94,4.91,4.89,4.45,4.74,NaN,t,3,0,3,

In [14]:
listings['name'] = listings.name.map(punc_lower)
listings.head()

,id,listing_url,name,description,neighborhood_overview,picture_url,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,amenities_upd
0,13913,https://www.airbnb.com/rooms/13913,holiday london db room let on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,"Islington, Greater London, United Kingdom",Islington,NaN,51.56861,-0.11270,Private room in rental unit,Private room,2,NaN,1 shared bath,1.0,NaN,"[""Hot water"", ""Heating"", ""Coffee maker"", ""Buil...",$65.00,1,29,1.0,1.0,29.0,29.0,1.0,29.0,NaN,t,26,56,86,361,2021-12-09,22,1,1,2010-08-18,2021-11-21,4.86,4.80,4.85,4.80,4.90,4.65,4.75,NaN,f,2,1,1,0,"[Hot_water, Heating, Coffee_maker, Building_st..."
1,15400,https://www.airbnb.com/rooms/15400,bright chelsea apartment chelsea,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,https://a0.muscache.com/pictures/428392/462d26...,"London, United Kingdom",Kensington and Chelsea,NaN,51.48780,-0.16813,Entire rental unit,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Hot water"", ""Heating"", ""Coffee maker"", ""TV w...",$75.00,3,50,3.0,3.0,50.0,50.0,3.0,50.0,NaN,t,0,0,0,4,2021-12-09,89,0,0,2009-12-21,2020-03-16,4.79,4.84,4.88,4.87,4.82,4.93,4.73,NaN,f,1,1,0,0,"[Hot_water, Heating, Coffee_maker, TV_with_sta..."
2,17402,https://www.airbnb.com/rooms/17402,superb bed bath wifi trendy,You'll have a wonderful stay in this superb mo...,"Location, location, location! You won't find b...",https://a0.muscache.com/pictures/39d5309d-fba7...,"London, Fitzrovia, United Kingdom",Westminster,NaN,51.52195,-0.14094,Entire rental unit,Entire home/apt,6,NaN,2 baths,3.0,3.0,"[""Hot water"", ""Heating"", ""Coffee maker"", ""Iron...",$265.00,4,365,2.0,4.0,365.0,365.0,4.0,365.0,NaN,t,4,34,60,295,2021-12-09,43,1,0,2011-03-21,2021-06-28,4.69,4.80,4.68,4.66,4.66,4.85,4.59,NaN,f,1,1,0,0,"[Hot_water, Heating, Coffee_maker, Iron, Dryer..."
3,17506,https://www.airbnb.com/rooms/17506,boutique chelsea fulham double bed star ensuite,Enjoy a chic stay in this elegant but fully mo...,Fulham is 'villagey' and residential – a real ...,https://a0.muscache.com/pictures/11901327/e63d...,"London, United Kingdom",Hammersmith and Fulham,NaN,51.47935,-0.19743,Private room in townhouse,Private room,2,NaN,1 private bath,1.0,1.0,"[""Shampoo"", ""Carbon monoxide alarm"", ""Hot wate...",$150.00,3,21,3.0,3.0,21.0,21.0,3.0,21.0,NaN,t,29,59,89,364,2021-12-09,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,2,0,2,0,"[Shampoo, Carbon_monoxide_alarm, Hot_water, He..."
4,25123,https://www.airbnb.com/rooms/25123,clean big room in london room,Big room with double bed/ clean sheets/ clean ...,Barnet is one of the largest boroughs in Londo...,https://a0.muscache.com/pictures/456905/a004b9...,"Barnet, England, United Kingdom",Barnet,NaN,51.57438,-0.21081,Private room in residential home,Private room,2,NaN,1.5 shared baths,1.0,NaN,"[""Shampoo"", ""Washer"", ""Heating"", ""Coffee maker...",$29.00,10,1120,10.0,10.0,1125.0,1125.0,10.0,1125.0,NaN,t,0,0,0,0,2021-12-09,129,0,0,2010-04-14,2020-03-15,4.78,4.70,4.94,4.91,4.89,4.45,4.74,NaN,t,3,0,3,0,"[Shampoo

In [15]:
listings['description'] = listings['description'].map(alphanumeric).map(punc_lower)
listings['neighborhood_overview'] = listings['neighborhood_overview'].map(alphanumeric).map(punc_lower)

In [18]:
listings

,id,listing_url,name,description,neighborhood_overview,picture_url,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights
0,13913,https://www.airbnb.com/rooms/13913,holiday london db room let on going,my bright double bedroom with a large window h...,finsbury park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,"Islington, Greater London, United Kingdom",Islington,NaN,51.568610,-0.112700,Private room in rental unit,Private room,2,NaN,1 shared bath,1.0,NaN,"[""Hot water"", ""Heating"", ""Coffee maker"", ""Buil...",$65.00,1,29,1.0
1,15400,https://www.airbnb.com/rooms/15400,bright chelsea apartment chelsea,lots of windows and light st luke s gardens ...,it is chelsea,https://a0.muscache.com/pictures/428392/462d26...,"London, United Kingdom",Kensington and Chelsea,NaN,51.487800,-0.168130,Entire rental unit,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Hot water"", ""Heating"", ""Coffee maker"", ""TV w...",$75.00,3,50,3.0
2,17402,https://www.airbnb.com/rooms/17402,superb bed bath wifi trendy,you ll have a wonderful stay in this superb mo...,location location location you won t find b...,https://a0.muscache.com/pictures/39d5309d-fba7...,"London, Fitzrovia, United Kingdom",Westminster,NaN,51.521950,-0.140940,Entire rental unit,Entire home/apt,6,NaN,2 baths,3.0,3.0,"[""Hot water"", ""Heating"", ""Coffee maker"", ""Iron...",$265.00,4,365,2.0
3,17506,https://www.airbnb.com/rooms/17506,boutique chelsea fulham double bed star ensuite,enjoy a chic stay in this elegant but fully mo...,fulham is villagey and residential – a real ...,https://a0.muscache.com/pictures/11901327/e63d...,"London, United Kingdom",Hammersmith and Fulham,NaN,51.479350,-0.197430,Private room in townhouse,Private room,2,NaN,1 private bath,1.0,1.0,"[""Shampoo"", ""Carbon monoxide alarm"", ""Hot wate...",$150.00,3,21,3.0
4,25123,https://www.airbnb.com/rooms/25123,clean big room in london room,big room with double bed clean sheets clean ...,barnet is one of the largest boroughs in londo...,https://a0.muscache.com/pictures/456905/a004b9...,"Barnet, England, United Kingdom",Barnet,NaN,51.574380,-0.210810,Private room in residential home,Private room,2,NaN,1.5 shared baths,1.0,NaN,"[""Shampoo"", ""Washer"", ""Heating"", ""Coffee maker...",$29.00,10,1120,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66636,53709893,https://www.airbnb.com/rooms/53709893,private bedroom crystal palace in high street,nan,nan,https://a0.muscache.com/pictures/61e601a3-948d...,NaN,Croydon,NaN,51.419345,-0.079681,Entire condominium (condo),Entire home/apt,6,NaN,1 bath,3.0,3.0,"[""Washer"", ""Carbon monoxide alarm"", ""Heating"",...",$120.00,2,365,2.0
66637,53710986,https://www.airbnb.com/rooms/53710986,chiswick park bed bath chiswick west london,beautiful modern top floor penthouse apartment...,nan,https://a0.muscache.com/pictures/prohost-api/H...,NaN,Ealing,NaN,51.502002,-0.276895,Entire condominium (condo),Entire home/apt,8,NaN,2.5 baths,3.0,4.0,"[""Heating"", ""Private entrance"", ""First aid kit...",$288.00,3,365,7.0
66638,53711668,https://www.airbnb.com/rooms/53711668,shabby chic beds bungalow in woodgreen,a mix of modern and antique decor with bed b...,nan,https://a0.muscache.com/pictures/miso/Hosting-...,NaN,Haringey,NaN,51.599308,-0.114763,Entire bungalow,Entire home/apt,4,NaN,2 baths,2.0,1.0,"[""Hot water"", ""Heating"", ""Coffee maker"", ""Priv...",$104.00,5,300,5.0
66639,53712786,https://www.airbnb.com/rooms/53712786,outstanding luxury notting hill gate,beautiful light and breathtaking flat situ...,a two minute walk from notting hill gate stati...,https://a0.muscache.com/pictures/67045be9-8f64...,"Greater London, England, United Kingdom",Kensington and Chelsea,NaN,51.510407,-0.193729,Entire rental unit,Entire home/apt,4,NaN,2.5 bat

### Amenities 

In [19]:
listings_sub = listings[['listing_url','name','description','price', 'amenities']]

In [20]:
listings_sub['amenities_upd'] = listings_sub['amenities'].astype(str).str.replace(r'\[|\]|,', '')

In [21]:
listings_sub['amenities_upd'] = listings_sub['amenities_upd'].astype(str).str.replace('_', ' ')

In [22]:
listings_sub['amenities_upd'] = listings_sub['amenities_upd'].map(alphanumeric).map(punc_lower)

In [24]:
listings_sub.head()

,listing_url,name,description,price,amenities,amenities_upd
0,https://www.airbnb.com/rooms/13913,holiday london db room let on going,my bright double bedroom with a large window h...,$65.00,"[""Hot water"", ""Heating"", ""Coffee maker"", ""Buil...",hot water heating coffee maker building...
1,https://www.airbnb.com/rooms/15400,bright chelsea apartment chelsea,lots of windows and light st luke s gardens ...,$75.00,"[""Hot water"", ""Heating"", ""Coffee maker"", ""TV w...",hot water heating coffee maker tv with ...
2,https://www.airbnb.com/rooms/17402,superb bed bath wifi trendy,you ll have a wonderful stay in this superb mo...,$265.00,"[""Hot water"", ""Heating"", ""Coffee maker"", ""Iron...",hot water heating coffee maker iron d...
3,https://www.airbnb.com/rooms/17506,boutique chelsea fulham double bed star ensuite,enjoy a chic stay in this elegant but fully mo...,$150.00,"[""Shampoo"", ""Carbon monoxide alarm"", ""Hot wate...",shampoo carbon monoxide alarm hot water ...
4,https://www.airbnb.com/rooms/25123,clean big room in london room,big room with double bed clean sheets clean ...,$29.00,"[""Shampoo"", ""Washer"", ""Heating"", ""Coffee maker...",shampoo washer heating coffee maker e...


### Clean $ currency

In [25]:
def clean_currency(x):
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', ''))
    return(x)

listings_sub['price'] = listings_sub['price'].apply(clean_currency).astype('string')

In [27]:
listings_sub['vector'] = listings_sub[['name','description','price', 'amenities_upd']].astype(str).apply(lambda x: ','.join(x),axis=1)

In [28]:
listings_sub['vector'][0]

'holiday london db room let on going,my bright double bedroom with a large window has a relaxed feeling  it comfortably fits one or two and is centrally located just two blocks from finsbury park  enjoy great restaurants in the area and easy access to easy transport tubes  trains and buses  babies and children of all ages are welcome  br    br    b the space  b  br   hello everyone  br    br   i m offering my lovely double bedroom in finsbury park area  zone    for let in a shared apartment   br   you will share the apartment with me and it is fully furnished with a self catering kitchen  two people can easily sleep well as the room has a queen size bed  i also have a travel cot for a baby for guest with small children   br    br   i will require a deposit up front as a security gesture on both our parts and will be given back to you when you return the keys   br    br   i trust anyone who will be responding to this add would treat my home with care and respect    br    br   best wishe

## Vectorization and recommendations

In [29]:
vectorizer = CountVectorizer(stop_words='english',ngram_range=(1, 2))
matrix = vectorizer.fit_transform(listings_sub['vector'])
matrix.shape

(66641, 760076)

In [30]:
#Save all files
pickle.dump(vectorizer,open('vectorizor.pkl','wb'))
pickle.dump(matrix,open('matrix.pkl','wb'))
pickle.dump(listings_sub,open('listings_sub.pkl','wb'))

In [31]:
query = vectorizer.transform(['I want an apartment close to london eye with 3 bedrooms'])

In [33]:
similarity = cosine_similarity(query, matrix)
top_5 = np.argsort(similarity[0])[-5:]
best_res = np.argmax(similarity[0])
listings_sub.loc[best_res]

listing_url                  https://www.airbnb.com/rooms/18138855
name                                     next london eye apartment
description              stunning apartment in the heart of london
price                                                       235.00
amenities        ["Shampoo", "Heating", "Essentials", "TV", "Wi...
amenities_upd     shampoo   heating   essentials   tv   wifi   ...
vector           next london eye apartment,stunning apartment i...
Name: 19605, dtype: object

In [35]:
top_5[0]
listings_sub.loc[[47028]]

,listing_url,name,description,price,amenities,amenities_upd,vector
47028,https://www.airbnb.com/rooms/39554990,apartment with bedrooms,apartment with bedrooms in london provides a...,1020.00,"[""Washer"", ""Elevator"", ""Heating"", ""Breakfast"",...",washer elevator heating breakfast ess...,"apartment with bedrooms,apartment with bed..."
